In [372]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from ydata_profiling import ProfileReport
import math
%matplotlib inline

4 Questions:

1 - How much money did you spend on marketing?
2 - How much money did you earn at the moment (on the 7th day)?
3 - How much money do we plan to earn in 1 year (2 billing of the annual subscription)?
4 - ROMI 7d and 1y (7-day and 1-year yields)?

Data:

In [373]:
annual_subscription_price = 89.99
monthly_subscription_price = 12.99
CR_from_trial_to_paid = 0.4
# Refund rate:
annual_refund_rate = 0.15
monthly_refund_rate = 0.03

total_number_of_subscriptions = 5000

share_of_annual_plan = 0.8

# Subscription retention:
annual_retention = 0.2
monthly_retention = 0.8

CPM = 15
CTR = 0.015
CR_click_to_install = 0.3
CR_install_to_purchase = 0.15

1 - How much money did you spend on marketing?

Obviously our marketing cost depends on Facebook ads, so let's begin to explore these costs.

Total number of installs:

In [374]:
total_number_of_installs = total_number_of_subscriptions / CR_install_to_purchase
total_number_of_installs = math.floor(total_number_of_installs)
total_number_of_installs

33333

Total clicks

In [375]:
total_number_of_clicks = total_number_of_installs / CR_click_to_install
total_number_of_clicks = math.floor(total_number_of_clicks)
total_number_of_clicks

111110

Total shows(impressions)

In [376]:
total_number_of_shows = total_number_of_clicks / CTR
total_number_of_shows = math.floor(total_number_of_shows)
total_number_of_shows

7407333

So, finally total price for the marketing:

In [377]:
total_price_for_marketing = total_number_of_shows * CPM / 1000.0
print('Total price for marketing: ',total_price_for_marketing, '$')

Total price for marketing:  111109.995 $


2 - How much money did you earn at the moment (on the 7th day)?

In that case we would not take into account the annual subscription because on the seventh day still has free trial
Taking in mind that - Share of annual plan with 7 days free: 80%
We can do the next steps:

In [378]:
share_of_month_plan = 1 - share_of_annual_plan
share_of_month_plan = round(share_of_month_plan, 1)
share_of_month_plan

0.2

I assumed that total_number_of_subscriptions_for_seven_days it is the number of subscriptions for the first 7 days.

Let's evaluate our revenue for the first seven days then

In [379]:
total_number_of_subscriptions_for_seven_days = 200
total_revenue_on_seventh_day = total_number_of_subscriptions_for_seven_days * share_of_month_plan * monthly_subscription_price
print('Total revenue for the first 7 days: ',total_revenue_on_seventh_day, '$')

Total revenue for the first 7 days:  519.6 $


3 - How much money do we plan to earn in 1 year (2 billing of the annual subscription)?

I assumed that total_number_of_subscriptions it is the number of subscriptions for the 2 years.
Also, all annual and month subscriptions were made in the first day of the year(for evaluating annual and monthtly subscription rebill second time in the beginning of the second year)

Annual retention rate analysis:
We would discover number of annual subscriptions in each year
I took into account Conversion rate from trial to paid in the first year
I assumed that the number of subscriptions in the first year is 3333

In [380]:
number_of_years = 2
first_percentage_annual = sum([pow(annual_retention, x) for x in range(number_of_years)])
list_of_annual_percentages = [first_percentage_annual]

for x in range(number_of_years - 1):
    list_of_annual_percentages.append(annual_retention)
total_number_of_annual_subscriptions = total_number_of_subscriptions * share_of_annual_plan

list_total_number_of_annual_subscriptions = []
temp_annual = total_number_of_annual_subscriptions
append_item_annual = math.floor(temp_annual / first_percentage_annual * CR_from_trial_to_paid)
list_total_number_of_annual_subscriptions.append(append_item_annual)
temp_annual = append_item_annual
for percentage in list_of_annual_percentages[1:]:
    append_item_annual = math.floor(temp_annual * percentage)
    list_total_number_of_annual_subscriptions.append(append_item_annual)
    temp_annual = append_item_annual
    

print(list_total_number_of_annual_subscriptions)
print(sum(list_total_number_of_annual_subscriptions))
# Basically 40% of 4000 annual payments = 1600

[1333, 266]
1599


Month retention rate analysis:
We would discover number of month subscriptions in each month

In [381]:
number_of_months = 23
first_percentage_month = sum([pow(monthly_retention, x) for x in range(number_of_months)])
list_of_month_percentages = [first_percentage_month]
for x in range(number_of_months - 1):
    list_of_month_percentages.append(monthly_retention)
total_number_of_month_subscriptions = total_number_of_subscriptions * share_of_month_plan

list_total_number_of_month_subscriptions = []
temp_month = total_number_of_month_subscriptions
append_item_month = math.floor(temp_month / list_of_month_percentages[0])
list_total_number_of_month_subscriptions.append(append_item_month)
temp_month = append_item_month
for percentage in list_of_month_percentages[1:]:
    append_item_month = math.floor(temp_month * percentage)
    list_total_number_of_month_subscriptions.append(append_item_month)
    temp_month = append_item_month
    
print(list_total_number_of_month_subscriptions)
print(sum(list_total_number_of_month_subscriptions))

[201, 160, 128, 102, 81, 64, 51, 40, 32, 25, 20, 16, 12, 9, 7, 5, 4, 3, 2, 1, 0, 0, 0]
963


Finally, the total revenue for the first day of the second year would be:

In [382]:
total_revenue_of_annual_subscriptions_for_second_year = sum([number_of_subs * annual_subscription_price for number_of_subs in list_total_number_of_annual_subscriptions]) * (1 - annual_refund_rate)

total_revenue_of_month_subscriptions_for_second_year = sum([number_of_subs * monthly_subscription_price for number_of_subs in list_total_number_of_month_subscriptions[:13]]) * (1 - monthly_refund_rate)

total_revenue_for_second_year = total_revenue_of_annual_subscriptions_for_second_year + total_revenue_of_month_subscriptions_for_second_year
print('Total revenue:', total_revenue_for_second_year, '$')

Total revenue: 134053.3881 $


4 - ROMI 7d and 1y (7-day and 1-year yields)?

First of all, we don't have marketing cost of the first 7 days, so I will assume that total number of subscriptions will be the same as I assumed previously evaluating 7d revenue 'total_number_of_subscriptions_for_seven_days = 200'

In [383]:
total_number_of_installs_for_seven_days = total_number_of_subscriptions_for_seven_days / CR_install_to_purchase
total_number_of_installs_for_seven_days = math.floor(total_number_of_installs_for_seven_days)

total_number_of_clicks_for_seven_days = total_number_of_installs_for_seven_days / CR_click_to_install
total_number_of_clicks_for_seven_days = math.floor(total_number_of_clicks_for_seven_days)

total_number_of_shows_for_seven_days = total_number_of_clicks_for_seven_days / CTR
total_number_of_shows_for_seven_days = math.floor(total_number_of_shows_for_seven_days)

total_price_for_marketing_for_seven_days = total_number_of_shows_for_seven_days * CPM / 1000.0
print('Total price for marketing: ',total_price_for_marketing_for_seven_days, '$')

Total price for marketing:  4443.0 $


Romi 7d:

In [384]:
Romi_7d = (total_revenue_on_seventh_day - total_price_for_marketing_for_seven_days) / total_price_for_marketing_for_seven_days * 100
print('ROMI 7 days:', Romi_7d, '%')

ROMI 7 days: -88.30519918973667 %


Romi 1y:

In [385]:
Romi_1y = (total_revenue_for_second_year - total_price_for_marketing) / total_price_for_marketing * 100
print('ROMI 1 year:', Romi_1y, '%')

ROMI 1 year: 20.649261211828886 %
